# Working with multiple images

In this Jupyter Notebook, we will see how to apply multiples geometric manipulations on a large number of images with the help of the `for` loops.

1. Clip images to the extent of the Region of Interest (ROI) and resample Scene Classification maps from 20m to 10m
2. Apply SCL to reflectances images to mask invalid pixels (clouds, shadows,...)
3. Compute NDVI at each date
4. Compute mean NDVI for each polygon


In [1]:
import glob, os
import numpy as np
import pandas as pd
import geopandas as gpd

import rasterio
import rasterio.mask
from rasterio.enums import Resampling
from rasterio import plot
from rasterio.plot import show

import matplotlib
import matplotlib.pyplot as plt

import rasterstats
from rasterstats import zonal_stats

from pathlib import Path
from IPython.display import display

print(f'Numpy : {np.__version__}')
print(f'Pandas : {pd.__version__}')
print(f'GeoPandas : {gpd.__version__}')
print(f'Rasterio : {rasterio.__version__}')
print(f'Rasterstats : {rasterstats.__version__}')
print(f'Matplotlib : {matplotlib.__version__}')

Numpy : 1.19.2
Pandas : 1.1.5
GeoPandas : 0.8.1
Rasterio : 1.1.0
Rasterstats : 0.14.0
Matplotlib : 3.3.4


## Set paths for input and output directories

Create directories if there are missing using `Path` and `mkdir`

In [2]:
grp_letter   = 'X'
student_name = 'ndeffense'

# When you are connected to the computer room
'''
vector_path = 'X:/data/VECTOR/'
raster_path = 'X:/data/RASTER/'
output_path = f'X:/GROUP_{grp_letter}/TP/{student_name}/DATA/'
'''

# When you are connected to your personnal computer
vector_path = '/Users/Nicolas/OneDrive - UCL/LBRAT2104/VECTOR/'
raster_path = '/Volumes/nbdid-sst-lbrat2104/data/RASTER/'
output_path = '/Users/Nicolas/OneDrive - UCL/LBRAT2104/Output/'


print(f'Vector input path are set to : {vector_path}')
print(f'Raster input path are set to : {raster_path}')
print(f'Output path are set to       : {output_path}')

# Directory to store all clipped images
clip_path = f'{output_path}IM_ROI/'

# Directory to store all clipped images with SCL
clip_scl_path = f'{output_path}IM_ROI_SCL/'

# Directory to store all NDVI images
ndvi_path = f'{output_path}NDVI_ROI/'

# Create directories if missing
Path(clip_path).mkdir(parents=True, exist_ok=True)
Path(clip_scl_path).mkdir(parents=True, exist_ok=True)
Path(ndvi_path).mkdir(parents=True, exist_ok=True)

Vector input path are set to : /Users/Nicolas/OneDrive - UCL/LBRAT2104/VECTOR/
Raster input path are set to : /Volumes/nbdid-sst-lbrat2104/data/RASTER/
Output path are set to       : /Users/Nicolas/OneDrive - UCL/LBRAT2104/Output/


## Set parameters
    
It is a good habit to set all parameters at the begining of the script to easily find them and modify them if needed.

In [3]:
# Resampling parameters
# ---------------------

upscale_factor = 2

method = 'nearest'

if method == 'nearest':
    resampling_method = Resampling.nearest
elif method == 'bilinear':
    resampling_method = Resampling.bilinear
elif method == 'cubic':
    resampling_method = Resampling.cubic


# Cloud mask parameters
# ---------------------

nodata_val = -10000

## Manipulate a large number of images

Create a *DataFrame* with `Pandas` using `glob` to store all informations from the SAFE directories you have uploaded on *Copernicus Open Access Hub*

***
**pandas**

`pandas` is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language
- A fast and efficient DataFrame object for data manipulation with integrated indexing;
- Tools for reading and writing data between in-memory data structures and different formats: CSV and text files, Microsoft Excel, SQL databases;
- Time series-functionality;
- Python with pandas is in use in a wide variety of academic and commercial domains, including Finance, Neuroscience, Economics, Statistics, Advertising, Web Analytics, and more

**glob**

The `glob` module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell, although results are returned in arbitrary order. No tilde expansion is done, but *, ?, and character ranges expressed with [] will be correctly matched.

`glob.glob()` : Return a possibly-empty list of path names that match *pathname*, which must be a string containing a path specification. *pathname* can be either absolute (like `/usr/src/Python-1.5/Makefile`) or relative (like `../../Tools/*/*.gif`)

**for loop**

In Python, a `for` loop is usually written as a loop over an iterable object (e.g. list, dataframe, array,...). This means you don’t need a counting variable to access items in the iterable. Sometimes, though, you do want to have a variable that changes on each loop iteration. Rather than creating and incrementing a variable yourself, you can use Python’s `enumerate()` to get a counter and the value from the iterable at the same time!

***


In [7]:
# Set path where your SAFE folders are
L2A_path = f'{raster_path}TP_2/'

# Get a list with all reflectance bands at 10m resolution
L2A_BANDS_10M = glob.glob(f'{L2A_path}*.SAFE/GRANULE/*/IMG_DATA/R10m/*B0*.jp2')

# Get a list with all reflectance bands at 20m resolution
L2A_BANDS_20M = glob.glob(f'{L2A_path}*.SAFE/GRANULE/*/IMG_DATA/R20m/*B0*.jp2')

# Get a list with all Scene Classification maps (SCL) products at 20m resolution
L2A_SCL_20M  = glob.glob(f'{L2A_path}*.SAFE/GRANULE/*/IMG_DATA/R20m/*SCL*.jp2')

L2A_all = L2A_BANDS_10M + L2A_SCL_20M


# Create an empty DataFrame using pandas with 4 columns : date, tile name, band number and image path

df = pd.DataFrame(columns=['date','tile','band','im_path'])

# Populate DataFrame using a for loop

for i, im_path in enumerate(L2A_all):
    
    im_path = im_path.replace('\\','/')      # If you are working in Windows, you may need to change '\' by '/'
    
    tile = os.path.basename(im_path)[1:1+5]  # Find the tile name inside the file name
    date = os.path.basename(im_path)[7:7+8]  # Find the date inside the file name
    
    if 'B0' in os.path.basename(im_path):
        band = os.path.basename(im_path)[-10:-8]  # Find the band number inside the file name
        
    elif 'SCL' in os.path.basename(im_path):
        band = '0'                               # If the file is the SCL, set the band number to 0
    
    # Add file infos to the dataframe
    df.loc[i,'date']    = date
    df.loc[i,'tile']    = tile
    df.loc[i,'band']    = band
    df.loc[i,'im_path'] = im_path


# Now that the for loop is finished, you can sort the newly created dataframe first by date, then by tile and lastly by band.

df = df.sort_values(by=['date','tile','band'])

# Display the entire DataFrame

pd.set_option('display.max_rows', 10)       # Maximum number of rows pandas to print
pd.set_option('display.max_columns', None)  # Maximum number of columns pandas to print
pd.set_option('display.max_colwidth', None) # Maximum width in characters of a column to print

display(df)

,date,tile,band,im_path
56,20200116,31UFS,0,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20200116T105309_N0213_R051_T31UFS_20200116T122813.SAFE/GRANULE/L2A_T31UFS_A014949_20200116T105304/IMG_DATA/R20m/T31UFS_20200116T105309_SCL_20m.jp2
16,20200116,31UFS,02,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20200116T105309_N0213_R051_T31UFS_20200116T122813.SAFE/GRANULE/L2A_T31UFS_A014949_20200116T105304/IMG_DATA/R10m/T31UFS_20200116T105309_B02_10m.jp2
17,20200116,31UFS,03,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20200116T105309_N0213_R051_T31UFS_20200116T122813.SAFE/GRANULE/L2A_T31UFS_A014949_20200116T105304/IMG_DATA/R10m/T31UFS_20200116T105309_B03_10m.jp2
18,20200116,31UFS,04,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20200116T105309_N0213_R051_T31UFS_20200116T122813.SAFE/GRANULE/L2A_T31UFS_A014949_20200116T105304/IMG_DATA/R10m/T31UFS_20200116T105309_B04_10m.jp2
19,20200116,31UFS,08,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20200116T105309_N0213_R051_T31UFS_20200116T122813.SAFE/GRANULE/L2A_T31UFS_A014949_20200116T105304/IMG_DATA/R10m/T31UFS_20200116T105309_B08_10m.jp2
...,...,...,...,...
64,20201218,31UFS,0,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20201218T104349_N0214_R008_T31UFS_20201218T124059.SAFE/GRANULE/L2A_T31UFS_A019768_20201218T104343/IMG_DATA/R20m/T31UFS_20201218T104349_SCL_20m.jp2
48,20201218,31UFS,02,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20201218T104349_N0214_R008_T31UFS_20201218T124059.SAFE/GRANULE/L2A_T31UFS_A019768_20201218T104343/IMG_DATA/R10m/T31UFS_20201218T104349_B02_10m.jp2
49,20201218,31UFS,03,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20201218T104349_N0214_R008_T31UFS_20201218T124059.SAFE/GRANULE/L2A_T31UFS_A019768_20201218T104343/IMG_DATA/R10m/T31UFS_20201218T104349_B03_10m.jp2
50,20201218,31UFS,04,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20201218T104349_N0214_R008_T31UFS_20201218T124059.SAFE/GRANULE/L2A_T31UFS_A019768_20201218T104343/IMG_DATA/R10m/T31UFS_20201218T104349_B04_10m.jp2


### Display a subset of the DataFrame

In [8]:
date_select = '20200212'

df_subset = df.loc[df['date'] == date_select]

print(f'Information for image at date : {date_select}')

display(df_subset)

Information for image at date : 20200212


,date,tile,band,im_path
57,20200212,31UFS,0,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20200212T104049_N0214_R008_T31UFS_20200213T134833.SAFE/GRANULE/L2A_T31UFS_A015335_20200212T104614/IMG_DATA/R20m/T31UFS_20200212T104049_SCL_20m.jp2
20,20200212,31UFS,02,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20200212T104049_N0214_R008_T31UFS_20200213T134833.SAFE/GRANULE/L2A_T31UFS_A015335_20200212T104614/IMG_DATA/R10m/T31UFS_20200212T104049_B02_10m.jp2
21,20200212,31UFS,03,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20200212T104049_N0214_R008_T31UFS_20200213T134833.SAFE/GRANULE/L2A_T31UFS_A015335_20200212T104614/IMG_DATA/R10m/T31UFS_20200212T104049_B03_10m.jp2
22,20200212,31UFS,04,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20200212T104049_N0214_R008_T31UFS_20200213T134833.SAFE/GRANULE/L2A_T31UFS_A015335_20200212T104614/IMG_DATA/R10m/T31UFS_20200212T104049_B04_10m.jp2
23,20200212,31UFS,08,/Volumes/nbdid-sst-lbrat2104/data/RASTER/TP_2/S2B_MSIL2A_20200212T104049_N0214_R008_T31UFS_20200213T134833.SAFE/GRANULE/L2A_T31UFS_A015335_20200212T104614/IMG_DATA/R10m/T31UFS_20200212T104049_B08_10m.jp2


## 1. Clip all images & resample to 10m if needed

With a few lines of code and the powerful libraries `rasterio` and `GeoPandas`, we will automatically crop/clip the Sentinel-2 images over our study area (ROI) and resample the Scene Classification maps from 20m to 10m.


In [9]:
# Read ROI as a GeoDataFrame
roi_file = f'{output_path}ROI/extent_roi_32631.shp'

print(f'ROI filename is : {roi_file}')

roi = gpd.read_file(roi_file)


# Make a for loop to get all images paths from dataframe and apply operations one after the other

for src_file in df['im_path']:
    
    filename = os.path.basename(src_file)[:-4]      # Get the image name without the extension (.tif)
    
    clipped_dst_file = f'{clip_path}{filename}_ROI.tif'  # Set the name of the new clipped image
    
    if not os.path.isfile(clipped_dst_file):     # If clipped image doesn't exist, create it
        
        # Open original file with rasterio
        src = rasterio.open(src_file, "r", driver='JP2OpenJPEG')
        
        # Clip original file with rasterio
        out_image, out_transform = rasterio.mask.mask(src, roi.geometry, crop=True)
        out_meta = src.meta

        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
        
        # Write clipped file
        dst = rasterio.open(clipped_dst_file, "w", **out_meta)
        dst.write(out_image)

        # Close rasterio objects
        src.close()
        dst.close()

        print(f'A new clipped raster file is created : {clipped_dst_file}')
    
        # ------------------------------------------------------------------- #
        # If the new created file is a SCL map, we also need to resample it ! #
        # ------------------------------------------------------------------- #

        if 'SCL_20m' in clipped_dst_file:

            # Create new name for the resampled SCL
            resampled_dst_file = f'{clipped_dst_file[:-11]}10m_ROI.tif'

            # Open clipped SCL
            src = rasterio.open(clipped_dst_file, "r", driver='GTiff')

            # Update the metadata
            profile = src.profile
            width   = src.width*upscale_factor
            height  = src.height*upscale_factor

            # Resample data to target shape
            data = src.read(
                out_shape=(
                    src.count,
                    int(src.height * upscale_factor),
                    int(src.width * upscale_factor)
                ),
                resampling=resampling_method
            )

            # Scale image transform
            transform = src.transform * src.transform.scale(
                (src.width / data.shape[-1]),
                (src.height / data.shape[-2])
            )

            profile.update(transform=transform, width=width, height=height)


            # Write resampled image
            dst = rasterio.open(resampled_dst_file, "w", **profile)
            dst.write(data)

            # Close rasterio objects
            src.close()
            dst.close()

            print(f'--> A new resampled raster file is created : {resampled_dst_file}')

ROI filename is : /Users/Nicolas/OneDrive - UCL/LBRAT2104/Output/ROI/extent_roi_32631.shp


## 2. Apply Scene Classification maps on all reflectances images

In [10]:
# Get a list with all reflectance bands clipped on the ROI
list_im_ROI = glob.glob(f'{clip_path}*_B*_ROI.tif')


for im_file in list_im_ROI:
    
    # Get date of image
    date = os.path.basename(im_file)[7:7+15]
    
    # Find SCL corresponding to the given reflectances image
    scl_file = glob.glob(f'{clip_path}*{date}*SCL_10m_ROI.tif')[0]
   
    # Change \\ to / in path if you areworking on Windows
    im_file  = im_file.replace('\\','/')
    scl_file = scl_file.replace('\\','/')
    
    # Get filename (only basename without path!) and create a new file name
    filename = os.path.basename(im_file)[:-4]
    im_scl_file = f'{clip_scl_path}{filename}_SCL.tif'
    
    if not os.path.isfile(im_scl_file):

        # Open reflectance image with rasterio and read it as a numpy array
        im_src = rasterio.open(im_file, "r", driver='GTiff')
        im = im_src.read(1)

        # Open SCL image with rasterio and read it as a numpy array
        scl_src = rasterio.open(scl_file, "r", driver='GTiff')
        scl = scl_src.read(1)
        
        # Update profile informations with the new metadata
        profile = scl_src.profile

        profile.update(dtype=rasterio.int16,  # Set to int16 it is lighter than float
                       count=1,               # We will write 1 band by file
                       nodata=nodata_val,     # Set nodata value in metadata
                       compress='lzw')        # Compression option)
        
        # Use SCL categorical values to mask invalid reflectance pixels
        # -------------------------------------------------------------
        
        scl = scl.astype(float)  # Convert scl array to float to be able to use "np.nan"
        
        # Set nan to categories corresponding to invalid pixels
        # Set 1 to categories corresponding to valid pixels
        
        # Be careful, you may have to change this! (e.g. if you are working on snow anomalies)

        scl[scl == 0] = np.nan    # No data
        scl[scl == 1] = np.nan    # Saturated or defective
        scl[scl == 2] = np.nan    # Dark area pixels
        scl[scl == 3] = np.nan    # Cloud shadows
        scl[scl == 4] = 1         # Vegetation
        scl[scl == 5] = 1         # Not vegetated
        scl[scl == 6] = 1         # Water
        scl[scl == 7] = 1         # Unclassified
        scl[scl == 8] = np.nan    # Cloud medium probability
        scl[scl == 9] = np.nan    # Cloud high probability
        scl[scl == 10] = np.nan   # Thin cirrus
        scl[scl == 11] = np.nan   # Snow
        
        im_scl = im*scl
        im_scl[np.isnan(im_scl)] = nodata_val  # Change np.nan by -10000
        
        # Convert image type from float to integer16 to reduce storage
        im_scl = im_scl.astype(np.int16)

        # Write reflectance with no data values
        im_scl_dst = rasterio.open(im_scl_file, "w", **profile)
        im_scl_dst.write(im_scl, 1)

        # Close all rasterio objects
        im_src.close()
        scl_src.close()
        im_scl_dst.close()

        print(f'A new raster with SCL is created : {im_scl_file}')
    

print("All your reflectances images with SCL are ready ! ")
    

All your reflectances images with SCL are ready ! 


## 3. Compute NDVI in your ROI at each date

In [11]:
# Get a list with all reflectance bands clipped on the ROI
list_im_red_ROI_SCL = glob.glob(f'{clip_scl_path}*_B04_10m_ROI_SCL.tif')

# Suppress error message if we divide by NaN
np.seterr(invalid='ignore')


for im_file_red in list_im_red_ROI_SCL:
    
    im_file_nir = im_file_red.replace('B04','B08')

    # Set the name of the new NDVI image based on the name of the RED image 
    im_file_ndvi = f'{ndvi_path}{os.path.basename(im_file_red).replace("B04","NDVI")}'

    if not os.path.isfile(im_file_ndvi):

        # Open RED band
        red_src = rasterio.open(im_file_red, 'r')
        red = red_src.read(1)

        # Open NIR band
        nir_src = rasterio.open(im_file_nir, 'r')
        nir = nir_src.read(1)

        #  Get a copy of the source dataset's profile
        profile = red_src.profile

        # Convert -10000 to np.nan before computing NDVI
        red = red.astype(float)
        red[red == nodata_val] = np.nan

        nir = nir.astype(float)
        nir[nir == nodata_val] = np.nan
        
        # Compute NDVI
        NDVI = (nir - red) / (nir + red) * 1000
        
        # Convert np.nan to -10000 to be able to store the data in integer16 (int16)
        NDVI[np.isnan(NDVI)] = nodata_val
        NDVI = NDVI.astype(np.int16)

        # Write NDVI image
        dst = rasterio.open(im_file_ndvi, "w", **profile)
        dst.write(NDVI,1)
        
        # Close all rasterio objects
        nir_src.close()
        red_src.close()
        dst.close()

        print(f'A new raster file is created : {im_file_ndvi}')

print("All your NDVI images are ready ! ")


All your NDVI images are ready ! 
